In [1]:
import numpy as np
import transformers
import accelerate
import vllm
import bitsandbytes
from vllm import LLM, SamplingParams
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
from collections import Counter
import subprocess
import json


from sentence_transformers import SentenceTransformer, util
#matplotlib.use('TkAgg')
#from awq import AutoAWQForCausalLM
#from transformers import AutoTokenizer

/home/Patrick/vllmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-08 09:41:51,297	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
"""
Multitask Benchmark datenset json

"""



import json
file_path = "./projects/question.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]
    #for line in f:
    #    data = json.loads(line)
    #    print(data)
df_mtconversation = pd.DataFrame(data)
df_mtconversation



,question_id,text,category
0,1,How can I improve my time management skills?,generic
1,2,What are the most effective ways to deal with ...,generic
2,3,What are the main differences between Python a...,generic
3,4,How can I increase my productivity while worki...,generic
4,5,Can you explain the basics of quantum computing?,generic
...,...,...,...
75,76,Write a script for a YouTube video exploring t...,writing
76,77,Compose an engaging travel blog post about a r...,writing
77,78,Write a captivating movie review for a recentl...,writing
78,79,Structure a podcast script for an episode disc...,writing


In [20]:
import json
file_path = "./projects/question.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]
    #for line in f:
    #    data = json.loads(line)
    #    print(data)
df_mtconversation = pd.DataFrame(data)
df_mtconversation

# Categories:
print(df_mtconversation.category.unique())


array(['generic', 'knowledge', 'roleplay', 'common-sense', 'fermi',
       'counterfactual', 'coding', 'math', 'writing'], dtype=object)

In [21]:
# Coding texts

coding_texts = df[df['category'] == 'coding']['text']
coding_texts.values

array(['Develop a C++ program that reads a text file line by line and counts the number of occurrences of a specific word in the file.',
       'Implement a Python function to find the longest common subsequence of two input strings using dynamic programming.',
       'Implement a regular expression in Python to validate an email address.',
       'Write a program to find the nth Fibonacci number using dynamic programming.',
       'Implement a binary search algorithm to find a specific element in a sorted array.',
       'Implement a queue data structure using two stacks in Python.',
       'Implement a program to find the common elements in two arrays without using any extra data structures.'],
      dtype=object)

In [9]:
def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power

## Hier mit simplem MT_Benchmark

In [33]:
import json
file_path = "./projects/question.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]
    #for line in f:
    #    data = json.loads(line)
    #    print(data)
df_mtconversation = pd.DataFrame(data)
df_mtconversation


print(df_mtconversation.category.unique())
coding_texts = df_mtconversation[df_mtconversation['category'] == 'coding']['text']
coding_texts.values

['generic' 'knowledge' 'roleplay' 'common-sense' 'fermi' 'counterfactual'
 'coding' 'math' 'writing']


array(['Develop a C++ program that reads a text file line by line and counts the number of occurrences of a specific word in the file.',
       'Implement a Python function to find the longest common subsequence of two input strings using dynamic programming.',
       'Implement a regular expression in Python to validate an email address.',
       'Write a program to find the nth Fibonacci number using dynamic programming.',
       'Implement a binary search algorithm to find a specific element in a sorted array.',
       'Implement a queue data structure using two stacks in Python.',
       'Implement a program to find the common elements in two arrays without using any extra data structures.'],
      dtype=object)

In [34]:
# Wie oft soll jeder einzelne Eingabeprompt genutzt werden? wichtig für Mean, std..

bootstrapping = 10

## MT_Bench mit normalem Model

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import time

model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m",
                                             #"tiiuae/falcon-mamba-7b",
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained ("facebook/opt-125m")


def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power


# Load dataset once and keep it ready for all experiments
def load_dataset(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

# Filter dataset by category
def filter_texts_by_category(df, category):
    return df[df['category'] == category]['text'].values




    

# Run the bootstrapping experiment for each text in a given category
def run_experiment_for_texts(texts, bootstrapping):


    #metriken:
    latencies = []
    energy_per_token = []
    throughputs= []
    generated_texts= []


    for i, text in enumerate(texts):
        """
        Texts: einzelne Prompts die zu einer Kategorie gehören: z.B. 
        coding

        ['Develop a C++ program that reads a text file line by line and counts the number of occurrences of a specific word in the file.',
       'Implement a Python function to find the longest common subsequence of two input strings using dynamic programming.',
       'Implement a regular expression in Python to validate an email address.',
       'Write a program to find the nth Fibonacci number using dynamic programming.',
       'Implement a binary search algorithm to find a specific element in a sorted array.',
       'Implement a queue data structure using two stacks in Python.',
       'Implement a program to find the common elements in two arrays without using any extra data structures.']
        """


        inputs = tokenizer(text, return_tensors="pt").to("cuda") #einzelner prompt 
        
        text_latencies = []                                 #latency für einen prompt der 10x durchlaufen wird   ------ darüber am ende Mean berechnen
        text_energy_per_token = []                          #energy per token eines einzelnen prompts der 10x durchlaufen wird ------ darüber am ende Mean berechnen
        text_throughput = []                                  #throughput ------ darüber am ende Mean berechnen
        text_generated = []                                   #generated text------ darüber am ende Mean berechnen

        for _ in range(bootstrapping):                           # einzelner prompt wird 10 mal durchlaufen
            power_start = get_gpu_power()                       #power consumption
            start_time = time.time()                            #start time

            # Generate output from the model
            output = model.generate(inputs['input_ids'], max_new_tokens=200, do_sample=True) #generating output

            end_time = time.time()                              #end time
            power_end = get_gpu_power()                        #power consumption           

            # Measure latency
            latency = end_time - start_time
            text_latencies.append(latency)


            # Calculate energy consumption (average power * time)
            avg_power = (power_start + power_end) / 2
            energy = avg_power * latency
            output_tokens = len(output[0])
            energy_token = energy / output_tokens if output_tokens > 0 else 0
            text_energy_per_token.append(energy_token)


            #througput
            throughput = output_tokens / latency
            text_throughput.append(throughput)

            # Decode the generated output text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            # Filter out the repetitive question from the generated text
            filtered_generated_text = generated_text.replace(text, "").strip()
            text_generated.append(filtered_generated_text)


        latencies.append(text_latencies)
        energy_per_token.append(text_energy_per_token)
        throughputs.append(text_throughput)
        generated_texts.append(text_generated)

    return latencies, energy_per_token, throughputs, generated_texts





# Store and collect metrics for each category
def collect_metrics_for_categories(df, categories, bootstrapping):
    category_metrics = {}

    for category in categories:                                                              #für jede Kategorie         
        print(f"Processing category: {category}")
        texts = filter_texts_by_category(df, category)                                      #df nach Kategorie filtern
        latencies, energy_per_token, throughputs, generated_texts= run_experiment_for_texts(texts, bootstrapping)    #hier wird die funktion run_experiment_for_texts aufgerufen

        # Store metrics for each category
        category_metrics[category] = {
            "latencies": latencies,
            "energy_per_token": energy_per_token,
            "throughput" : throughputs,
            "generated_texts": generated_texts
        }

    return category_metrics





# Load the dataset
file_path = "./projects/question.jsonl"
bootstrapping = 10  # Number of iterations for each prompt

df_mtconversation = load_dataset(file_path)

# Define the categories to test
categories = ['generic', 'knowledge', 'roleplay', 'common-sense', 'fermi', 'counterfactual', 'coding', 'math', 'writing']   #sind alle kategorien die es gibt im MT_Bench datenset

# Collect metrics for each category
metrics = collect_metrics_for_categories(df_mtconversation, categories, bootstrapping)

# Plot results for comparison
#plot_energy_vs_latency(metrics, categories)


/home/Patrick/vllmenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing category: generic
Processing category: knowledge
Processing category: roleplay
Processing category: common-sense
Processing category: fermi
Processing category: counterfactual
Processing category: coding
Processing category: math
Processing category: writing


In [5]:
# Plot the energy consumption per token comparison
def plot_energy_vs_latency(metrics, categories):
    for category in categories:
        category_data = metrics[category]
        energy_per_token = category_data["energy_per_token"]
        latencies = category_data["latencies"]
        response_sim = category_data["response_similiarity"]

        plt.figure(figsize=(10, 5))

        # Plot energy per token
        plt.subplot(1, 2, 1)
        plt.plot([sum(x)/len(x) for x in energy_per_token], marker='o', color='blue', label='Energy per Token (J)')
        plt.title(f"Energy per Token for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('Energy (J)')
        plt.legend()

        # Plot latencies
        plt.subplot(1, 2, 2)
        plt.plot([sum(x)/len(x) for x in latencies], marker='o', color='green', label='latency')
        plt.title(f"latency for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('simScore')
        plt.legend()

        plt.tight_layout()
        plt.show()


In [6]:
#plot_energy_vs_latency(metrics, categories)

## MT_Bench mit quantized Model

In [7]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Specify the quantization type
    bnb_4bit_use_double_quant=True,  # Use double quantization if needed
    bnb_4bit_compute_dtype=torch.float16  # Specify computation dtype
)


# Configure 4-bit quantization
#quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }
# Load the model and tokenizer with quantization
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m",
                                             #"tiiuae/falcon-mamba-7b",
                                             quantization_config=quant_config,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained ("facebook/opt-125m")

/home/Patrick/vllmenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import time



# Load a pre-trained sentence transformer model
modelSimiliarity = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def calculate_semantic_similarity(response_1, response_2):
    # Get embeddings for both responses
    embedding_1 = modelSimiliarity.encode(response_1, convert_to_tensor=True)
    embedding_2 = modelSimiliarity.encode(response_2, convert_to_tensor=True)

    # Compute cosine similarity
    cosine_sim = util.pytorch_cos_sim(embedding_1, embedding_2)
    return cosine_sim.item()

def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power


# Load dataset once and keep it ready for all experiments
def load_dataset(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

# Filter dataset by category
def filter_texts_by_category(df, category):
    return df[df['category'] == category]['text'].values

# Run the bootstrapping experiment for each text in a given category
def run_experiment_for_texts(texts, bootstrapping):
    latencies = []
    energy_per_token = []
    throughputs= []
    generated_texts= []

    for i, text in enumerate(texts):
        inputs = tokenizer(text, return_tensors="pt").to("cuda")
        
        text_latencies = []
        text_energy_per_token = []
        text_throughput = []
        text_generated = []
        response_similiarity = []

        for _ in range(bootstrapping):
            power_start = get_gpu_power()
            start_time = time.time()

            # Generate output from the model
            output = model.generate(inputs['input_ids'], max_new_tokens=200, do_sample=True)

            end_time = time.time()
            power_end = get_gpu_power()

            # Measure latency
            latency = end_time - start_time
            text_latencies.append(latency)


            # Calculate energy consumption (average power * time)
            avg_power = (power_start + power_end) / 2
            energy = avg_power * latency
            output_tokens = len(output[0])
            energy_token = energy / output_tokens if output_tokens > 0 else 0
            text_energy_per_token.append(energy_token)


            #througput
            throughput = output_tokens / latency
            text_throughput.append(throughput)

            # Decode the generated output text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            # Filter out the repetitive question from the generated text
            filtered_generated_text = generated_text.replace(text, "").strip()
            text_generated.append(filtered_generated_text)

        # measuring the similiarity    
        n = len(text_generated)
        similarity_scores = []  # To store similarity scores

        for i in range(n - 1):  # Outer loop goes up to n-2
            for j in range(i + 1, n):  # Inner loop starts from i+1 and goes to n-1
                similarity = calculate_semantic_similarity(text_generated[i], text_generated[j])
                similarity_scores.append(similarity)  # Store only the pairwise similarity scores

        # Step 2: Calculate average and variance of similarities
        average_similarity_Encoder = np.mean(similarity_scores)
        std_similarity_Encoder = np.std(similarity_scores)
        response_similiarity.append((average_similarity_Encoder, std_similarity_Encoder))


        latencies.append(text_latencies)
        energy_per_token.append(text_energy_per_token)
        throughputs.append(text_throughput)
        generated_texts.append(text_generated)

    return latencies, energy_per_token, throughputs, generated_texts, response_similiarity

# Store and collect metrics for each category
def collect_metrics_for_categories(df, categories, bootstrapping):
    category_metrics = {}

    for category in categories:
        print(f"Processing category: {category}")
        texts = filter_texts_by_category(df, category)
        latencies, energy_per_token, throughputs, generated_texts, response_similiarity = run_experiment_for_texts(texts, bootstrapping)

        # Store metrics for each category
        category_metrics[category] = {
            "latencies": latencies,
            "energy_per_token": energy_per_token,
            "throughput" : throughputs,
            "generated_texts": generated_texts,
            "response_similiarity": response_similiarity
        }

    return category_metrics





# Load the dataset
file_path = "./projects/question.jsonl"
bootstrapping = 10  # Number of iterations for each prompt

df_mtconversation = load_dataset(file_path)

# Define the categories to test
categories = ['generic', 'knowledge', 'roleplay', 'common-sense', 'fermi', 'counterfactual', 'coding', 'math', 'writing']

# Collect metrics for each category
metricsquant = collect_metrics_for_categories(df_mtconversation, categories, bootstrapping)

# Plot results for comparison
#plot_energy_vs_latency(metrics, categories)


Processing category: generic
Processing category: knowledge
Processing category: roleplay
Processing category: common-sense
Processing category: fermi
Processing category: counterfactual
Processing category: coding


In [ ]:
# Plot the energy consumption per token comparison
def plot_energy_vs_latency(metrics, categories):
    for category in categories:
        category_data = metrics[category]
        energy_per_token = category_data["energy_per_token"]
        latencies = category_data["latencies"]
        response_sim = category_data["response_similiarity"]

        plt.figure(figsize=(10, 5))

        # Plot energy per token
        plt.subplot(1, 2, 1)
        plt.plot([sum(x)/len(x) for x in energy_per_token], marker='o', color='blue', label='Energy per Token (J)')
        plt.title(f"Energy per Token for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('Energy (J)')
        plt.legend()

        # Plot latencies
        plt.subplot(1, 2, 2)
        plt.plot([sum(x)/len(x) for x in latencies], marker='o', color='green', label='latency')
        plt.title(f"latency for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('simScore')
        plt.legend()

        plt.tight_layout()
        plt.show()


## MT_Bench with vLLM


### Das funktioniert noch nicht mit vLLM!!!  out of memory error


In [2]:
import numpy as np
import transformers
import accelerate
import vllm
import bitsandbytes
from vllm import LLM, SamplingParams
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
from collections import Counter
import subprocess
import json


from sentence_transformers import SentenceTransformer, util
#matplotlib.use('TkAgg')
#from awq import AutoAWQForCausalLM
#from transformers import AutoTokenizer

/home/Patrick/vllmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-26 16:22:31,750	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:



def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power


tokenizer = AutoTokenizer.from_pretrained ("facebook/opt-125m")
#llm = LLM("tiiuae/falcon-7b",dtype = torch.float16 ,trust_remote_code=True)
llm = LLM("facebook/opt-125m",trust_remote_code=True)
# Prepare sampling parameters and prompt
sampling_params = SamplingParams(temperature=0.5, max_tokens=200)
#prompt = "Generate a python code that accepts a list of numbers and returns the sum."



def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power


# Load dataset once and keep it ready for all experiments
def load_dataset(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

# Filter dataset by category
def filter_texts_by_category(df, category):
    return df[df['category'] == category]['text'].values

# Run the bootstrapping experiment for each text in a given category
def run_experiment_for_texts(texts, bootstrapping):
    latencies = []
    energy_per_token = []
    throughputs= []
    generated_texts= []

    for i, text in enumerate(texts):
        #inputs = tokenizer(text, return_tensors="pt").to("cuda")
        
        text_latencies = []
        text_energy_per_token = []
        text_throughput = []
        text_generated = []


        for _ in range(bootstrapping):
            power_start = get_gpu_power()
            start_time = time.time()



            #prompts = inputs['input_ids'].tolist()
            # Generate output from the model
            response = llm.generate(text, sampling_params)



            end_time = time.time()
            power_end = get_gpu_power()
            # Measure latency
            latency = end_time - start_time
            text_latencies.append(latency)


            # Calculate energy consumption (average power * time)
            avg_power = (power_start + power_end) / 2
            energy = avg_power * latency
            output_tokens = len(output[0])
            energy_token = energy / output_tokens if output_tokens > 0 else 0
            text_energy_per_token.append(energy_token)


            #througput
            throughput = output_tokens / latency
            text_throughput.append(throughput)

            # Decode the generated output text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            # Filter out the repetitive question from the generated text
            filtered_generated_text = generated_text.replace(text, "").strip()
            text_generated.append(filtered_generated_text)

        latencies.append(text_latencies)
        energy_per_token.append(text_energy_per_token)
        throughputs.append(text_throughput)
        generated_texts.append(text_generated)

    return latencies, energy_per_token, throughputs, generated_texts

# Store and collect metrics for each category
def collect_metrics_for_categories(df, categories, bootstrapping):
    category_metrics = {}

    for category in categories:
        print(f"Processing category: {category}")
        texts = filter_texts_by_category(df, category)
        latencies, energy_per_token, throughputs, generated_texts = run_experiment_for_texts(texts, bootstrapping)

        # Store metrics for each category
        category_metrics[category] = {
            "latencies": latencies,
            "energy_per_token": energy_per_token,
            "throughput" : throughputs,
            "generated_texts": generated_texts
        }

    return category_metrics





# Load the dataset
file_path = "./projects/question.jsonl"
bootstrapping = 10  # Number of iterations for each prompt

df_mtconversation = load_dataset(file_path)

# Define the categories to test
categories = ['generic', 'knowledge', 'roleplay', 'common-sense', 'fermi', 'counterfactual', 'coding', 'math', 'writing']

# Collect metrics for each category
metricsquant = collect_metrics_for_categories(df_mtconversation, categories, bootstrapping)

/home/Patrick/vllmenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


INFO 09-26 16:22:36 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=facebook/opt-125m, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_output_proc=True)
INFO 09

/home/Patrick/vllmenv/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/Patrick/vllmenv/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# MMLU 

In [ ]:
#Science, Technology, Engineering, Mathematics = stem
stem = ["clinical_knowledge",
"medical_genetics", 
"high_school_physics",
"virology",
"high_school_biology",
"abstract_algebra",
"professional_medicine",
"nutrition",
"machine_learning",
"anatomy",
"college_medicine",
"college_chemistry",
"elementary_mathematics",
"human_aging",
"college_mathematics",
"high_school_statistics",
"high_school_mathematics",
"high_school_computer_science",
"conceptual_physics",
"high_school_chemistry",
"college_physics",
"electrical_engineering",
"astronomy",
"college_biology",
"computer_security"]

humanities= ["high_school_european_history",
"high_school_us_history",
"high_school_world_history",
"philosophy",
"global_facts",
"security_studies",
"prehistory",
"high_school_government_and_politics",
"logical_fallacies",
"international_law",
"jurisprudence",
"world_religions",
"us_foreign_policy",
"moral_scenarios",
"moral_disputes"
]

sociology = ["sociology",
"professional_psychology",
"high_school_psychology",
"human_sexuality"]

others = ["business_ethics",
"high_school_microeconomics",
"econometrics",
"professional_accounting",
"public_relations",
"marketing",
"professional_law",
"management",
"miscellaneous",
"high_school_macroeconomics"]

In [ ]:
import pandas as pd
from datasets import load_dataset

def convert_to_dataframe(categories):
    category_dataframes = {}  # Dictionary to store DataFrames for each category
    
    for category in categories:
        print("Loading Data for category: ", category)
        
        # Load the dataset for the given category
        mmlu_dataset = load_dataset("lukaemon/mmlu", category, split='validation', trust_remote_code=True)
        
        # Create a DataFrame for the current category
        df_category = pd.DataFrame({
            'input': mmlu_dataset['input'],  # The question or prompt
            'A': mmlu_dataset['A'],          # Option A
            'B': mmlu_dataset['B'],          # Option B
            'C': mmlu_dataset['C'],          # Option C
            'D': mmlu_dataset['D'],          # Option D
            'target': mmlu_dataset['target'] # The correct answer (e.g., 'A', 'B', 'C', 'D')
        })
        
        # Store the DataFrame in the dictionary, with the category as the key
        category_dataframes[category] = df_category
    
    return category_dataframes

# Access a DataFrame for a specific category
#print(category_dfs.head())  # Example for checking 'high_school_biology'


In [ ]:
"""
stem, humanities ..etc
"""

# Example list of categories
categories = ['high_school_biology', 'abstract_algebra', 'professional_medicine', 'nutrition']

# Call the function and get the dictionary of DataFrames
category_dfs = convert_to_dataframe(categories)

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import time
from datasets import load_dataset

#cataegory = "sociology"
# Load the MMLU dataset
#def load_mmlu_dataset():
#    mmlu_dataset = load_dataset("lukaemon/mmlu",category, split='validation',trust_remote_code=True)
#    return mmlu_dataset



# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

def get_gpu_power(gpu_index=1): #measuring for gpu1
    """Fetches the current power consumption of the GPU using nvidia-smi."""
    result = subprocess.run(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits', '-i',str(gpu_index)], 
                            stdout=subprocess.PIPE, text=True)
    power = float(result.stdout.strip())  # Power in watts
    return power

def convert_to_dataframe(categories):
    category_dataframes = {}  # Dictionary to store DataFrames for each category
    
    for category in categories:
        print("Loading Data for category: ", category)
        
        # Load the dataset for the given category
        mmlu_dataset = load_dataset("lukaemon/mmlu",category, split='validation',trust_remote_code=True)
        #print(type(mmlu_dataset))
        # Create a DataFrame for the current category
        df_category = pd.DataFrame({
            'input': mmlu_dataset['input'],  # The question or prompt
            'A': mmlu_dataset['A'],          # Option A
            'B': mmlu_dataset['B'],          # Option B
            'C': mmlu_dataset['C'],          # Option C
            'D': mmlu_dataset['D'],          # Option D
            'target': mmlu_dataset['target'] # The correct answer (e.g., 'A', 'B', 'C', 'D')
        })
        
        # Store the DataFrame in the dictionary, with the category as the key
        category_dataframes[category] = df_category
    print("loading_ data finish")
    return category_dataframes

# Filter dataset by category
#def filter_texts_by_category(df, category):
#    return df[df['category'] == category]['text'].values

# Filter dataset by subject category (e.g., 'high_school_biology', 'abstract_algebra')
def filter_dict_by_category(df, category):
    return df[category] 


import time

# Run the bootstrapping experiment for multiple-choice questions in a given category
def run_experiment_for_texts(datadictionary,categories, bootstrapping ):
    category_latencies = []
    category_energy_per_token = []
    
    
    #print(categories)
    category_accuracy= []



    # task in one category
    for category in categories:
        data = datadictionary[category]#filter_dict_by_category(datadictionary, category)
        question_text = data['input'].values
        choices = [data['A'].values, data['B'].values, data['C'].values, data['D'].values]
        correct_answer = data['target'].values
        
        task_accuracy= []
        task_latencies = []
        task_energy_per_token = []
        #print(" type question_text: ", question_text)
        #print("len question_text: ", len(question_text))
        print("processing category: ", category)


        # Prompts of one tasks
        for i, tasks in enumerate (question_text):



            print("i : ", i)
            # Concatenate question with options for LLM input
            full_input = f"Question: {question_text[i]}\nA) {choices[0][i]}\nB) ,{choices[1][i]}\nC) ,{choices[2][i]}\nD) ,{choices[3][i]}"
            #print("full input: ", full_input)
            inputs = tokenizer(full_input, return_tensors="pt").to("cuda")  # Prepare input tensors

            text_latencies = []
            text_energy_per_token = []
            correct_predictions = 0  # To calculate accuracy




            # Prompt Bootstrapping 
            for _ in range(bootstrapping):




                print(_)
                power_start = get_gpu_power()  # Assuming a function to get GPU power
                start_time = time.time()

                # Generate the model's response
                output = model.generate(inputs['input_ids'], max_new_tokens=200, do_sample=False)  # Adjust tokens if necessary

                end_time = time.time()
                power_end = get_gpu_power()

                # Measure latency
                latency = end_time - start_time
                text_latencies.append(latency)

                # Calculate energy consumption
                avg_power = (power_start + power_end) / 2
                energy = avg_power * latency

                # Token count from output (assuming a tensor output)
                output_tokens = output[0].shape[0]
                energy_token = energy / output_tokens if output_tokens > 0 else 0
                text_energy_per_token.append(energy_token)

                # Decode the model's generated answer (you might need to adjust based on model output format)
                generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

                # Check if the model's generated answer matches the correct answer
                if correct_answer[i] in generated_text:
                    correct_predictions += 1

            task_latencies.append(np.mean(text_latencies))
            task_energy_per_token.append(np.mean(text_energy_per_token))
            accuracy = correct_predictions / bootstrapping 
            task_accuracy.append(accuracy)
    category_accuracy.append(np.array(task_accuracy)/len(question_text))
    return task_latencies, task_energy_per_token, category_accuracy


# Store and collect metrics for each category
# Collect metrics for each category
def collect_metrics_for_categories(df, categories, bootstrapping):
    category_metrics = {}

    """    for category in categories:
            print(f"Processing category: {category}")
            texts = filter_texts_by_category(df, category)

            if texts.empty:
                print(f"No texts found for category {category}")
                continue
    """
    latencies, energy_per_token, accuracy = run_experiment_for_texts(data_dict, categories, bootstrapping)

    # Store metrics for each category
    category_metrics[category] = {
        "latencies": latencies,
        "energy_per_token": energy_per_token,
        "accuracy": accuracy
    }

    return category_metrics

# Plot the energy consumption per token comparison
def plot_energy_vs_latency(metrics, categories):
    for category in categories:
        category_data = metrics[category]
        energy_per_token = category_data["energy_per_token"]
        latencies = category_data["latencies"]

        plt.figure(figsize=(10, 5))

        # Plot energy per token
        plt.subplot(1, 2, 1)
        plt.plot([sum(x)/len(x) for x in energy_per_token], marker='o', color='blue', label='Energy per Token (J)')
        plt.title(f"Energy per Token for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('Energy (J)')
        plt.legend()

        # Plot latencies
        plt.subplot(1, 2, 2)
        plt.plot([sum(x)/len(x) for x in latencies], marker='o', color='green', label='Latency (s)')
        plt.title(f"Latency for {category}")
        plt.xlabel('Iteration')
        plt.ylabel('Latency (s)')
        plt.legend()

        plt.tight_layout()
        plt.show()


    #plot_energy_vs_latency(metrics, categories)


In [ ]:
"""
Processing data

"""


categories = stem
categories = sociology  #sociology is shorter / less prompts
#mmlu_dataset = load_mmlu_dataset()
data_dict = convert_to_dataframe(categories)
print("data_dict geladen")


In [ ]:
"""
Run Experiments

"""


bootstrapping = 3  # Number of iterations for each prompt

# Collect metrics for each category
metrics = collect_metrics_for_categories(data_dict, categories, bootstrapping)
